# Pit Matrix Example

In [1]:
# standard imports
import numpy as np

#database imports
from snowexsql.db import get_db
from snowexsql.data import SiteData, ImageData, LayerData, PointData
from snowexsql.conversions import query_to_geopandas
import datetime

## Distinguish pits by vegetation coverage: 
- treeless (0% tree cover)
- sparse (1-30% tree cover)
- dense (31-100% tree cover)

*vegetation classes assigned based on optical imagery: tree density map, Nov. 2010 WorldView-2 Imagery

In [2]:
def parse_veg_class(site_id):
    
    # Classifying by vegetation coverage 
    veg_class = {'treeless':[1, 2, 3], 'sparse':[4, 5, 6], 'dense':[7, 8, 9]}
   
    vclass = None 
    
    class_id = site_id[0]
    
    if class_id.isnumeric():
        class_id = int(class_id)

        for k,v in veg_class.items():

            if class_id in v:
                vclass = k 
                
    return vclass 

### Distinguish pits by snow depth classes: 
- shallow (<90cm)
- medium (90-122cm)
- deep (>122cm)

*depth classes assigned based on 2017 snow depth lidar data

In [3]:
def parse_depth_class(site_id):
    # Classifying by expected depth 
    depth_class = {'shallow':[1, 4, 7], 'medium':[2, 5, 8], 'deep':[3, 6, 9]} 
   
    dclass = None 
    
    class_id = site_id[0]
    
    if class_id.isnumeric(): #for the outlier TS site
        class_id = int(class_id) #cast as integer

        for k,v in depth_class.items(): #for the key, value pairs in the dict listed above:

            if class_id in v:
                dclass = k 

    return dclass 

In [14]:
# load the database
db_name = 'snow:hackweek@52.32.183.144/snowex'
engine, session = get_db(db_name)

result = session.query(LayerData.type).distinct().all()

qry = session.query(LayerData).filter(LayerData.type=='density')

# Form our dataframe from the query 
df = query_to_geopandas(qry, engine)
df['value'] = df['value'].astype(float) #cast the value as a float (they are strings)
 
# parse snow pit data by the veg/depth matrix
df['veg_class'] = [parse_veg_class(i) for i in df['site_id']] #run the parse_veg function for every site_id
df['depth_class'] = [parse_depth_class(i) for i in df['site_id']] #run the parse_depth funciton for every site_id

# # Show off our df 
# df.plot()

df.columns
col_list = ['site_name', 'date', 'id', 'instrument', 'type', 'units', 'surveyors', 'latitude',
       'longitude', 'geom','depth', 'site_id', 'value', 'veg_class', 'depth_class']
df = df[col_list]
df

,site_name,date,id,instrument,type,units,surveyors,latitude,longitude,geom,depth,site_id,value,veg_class,depth_class
0,Grand Mesa,2020-01-30,4272,None,density,None,None,39.039987,-108.191917,POINT (743040.000 4324967.000),75.0,9C17,124.5,dense,deep
1,Grand Mesa,2020-01-30,4273,None,density,None,None,39.039987,-108.191917,POINT (743040.000 4324967.000),65.0,9C17,207.0,dense,deep
2,Grand Mesa,2020-01-30,4274,None,density,None,None,39.039987,-108.191917,POINT (743040.000 4324967.000),55.0,9C17,230.0,dense,deep
3,Grand Mesa,2020-01-30,4275,None,density,None,None,39.039987,-108.191917,POINT (743040.000 4324967.000),45.0,9C17,258.5,dense,deep
4,Grand Mesa,2020-01-30,4276,None,density,None,None,39.039987,-108.191917,POINT (743040.000 4324967.000),35.0,9C17,258.0,dense,deep
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2817,Grand Mesa,2020-01-29,2356,None,density,None,None,39.014367,-108.141578,POINT (747487.000 4322259.000),42.0,3S47,256.5,treeless,deep
2818,Grand Mesa,2020-01-29,2357,None,density,None,None,39.014367,-108.141578,POINT (747487.000 4322259.000),32.0,3S47,281.0,treeless,deep
2819,Grand Mesa,2020-01-29,2358,None,density,None,None,39.014367,-108.141578,POINT (747487.000 4322259.000),22.0,3S47,299.5,treeless,deep
2820,Grand Mesa,2020-01-29,2359,None,density,None,None,39.014367,-108.141578,POINT (747487.000 4322259.000),12.0,3S47,262.5,treeless,deep


In [30]:
df.depth

0       75.0
1       65.0
2       55.0
3       45.0
4       35.0
        ... 
2817    42.0
2818    32.0
2819    22.0
2820    12.0
2821    79.0
Name: depth, Length: 2822, dtype: float64

In [42]:
# These are all of the unique site ids 
len(df['site_id'].unique())

153

In [56]:
#make df with only 153 rows using site_id.unique
gb = df.groupby(['site_id', 'veg_class'])
#gb = df.groupby('site_id')
gb['site_name'].count().groupby("veg_class").count()


veg_class
dense       50
sparse      39
treeless    60
Name: site_name, dtype: int64

In [ ]:
# d = df[['veg_class', 'depth_class']].groupby(df['site_id']).groups
# gb = df.groupby(['veg_class', 'site_id']).count()
# gb = df.groupby(['site_id']).count()
df['veg_class'].groupby('veg_class').count()

# print(gb)

In [23]:
df['site_id'].unique().groupby('veg_class').count()


AttributeError: 'numpy.ndarray' object has no attribute 'groupby'

In [24]:
df[['veg_class', 'site_id']].groupby('veg_class').count() #table I like!

,site_id
veg_class,
dense,923
sparse,801
treeless,1017


In [ ]:
df[['veg_class', 'counter']].groupby("veg_class").sum()

In [ ]:
df[['veg_class', 'depth']].groupby("veg_class").mean()

In [ ]:
## Plot depth classes
df[['depth_class', 'counter']].groupby("depth_class").sum()

In [ ]:
df[['depth_class', 'depth']].groupby("depth_class").mean()

In [ ]:
unique_pit_id = df['site_id'] + df['date'].astype(str)
df['unique_pit_id'] = unique_pit_id

# groups = df[['unique_pit_id', 'veg_class', 'counter']].groupby('veg_class').groups
print(df.loc[df.veg_class == "treeless"].groupby(unique_pit_id).mean()['depth'].mean())
print(df.loc[df.veg_class == "sparse"].groupby(unique_pit_id).mean()['depth'].mean())
print(df.loc[df.veg_class == "dense"].groupby(unique_pit_id).mean()['depth'].mean())



In [ ]:
len(df.loc[df.veg_class == "treeless"]["unique_pit_id"].unique())


In [ ]:
len(df.loc[df.veg_class == "sparse"]["unique_pit_id"].unique())


In [ ]:
len(df.loc[df.veg_class == "dense"]["unique_pit_id"].unique())


In [ ]:
# load the database
db_name = 'snow:hackweek@52.32.183.144/snowex'
engine, session = get_db(db_name)

result = session.query(SiteData).distinct().all()


qry = session.query(SiteData)

# # Form our dataframe from the query 
df = query_to_geopandas(qry, engine)
# df['value'] = np.float64(df.value)   # these come out as strings ... 
# df['counter'] = 1 

# df['veg_class'] = [parse_veg_class(i) for i in df['site_id']]
# df['depth_class'] = [parse_depth_class(i) for i in df['site_id']]

# # Show off our df 
df


In [ ]:
df['veg_class'].plot.hist()

PLOT: #1 Density

  - depth (cm) vs. density (kg/m3) for snow depth classes
  - 3 colors: shallow (r), medium (g), deep (b)
  - vertical bars to show the 10cm measurement increments (see ../images)  
  - time period -- (hmm, all? single day? 1 week avg?)

PLOT: #2 Temperature

  - depth (cm) vs. temperature (C) for snow depth classes
  - 3 colors: shallow (r), medium (g), deep (b)
  - x-axis is -10 to 0 (see ../images)  
  - time period -- (hmm, all? single day? 1 week avg?)
  - could plot all pits with light gray line (or group by depth classes in a light/transparent color) and then take the avg. of 'shallow', 'medium', 'deep' and plot with a bold color line

PLOT: #3
  - do something with stratigraphy here....still thinking on that...

In [ ]:
# site_id = '5N34'
# dclass = None 
# class_id = site_id[0]

# # (k for k,v in dict.iteritems() if v == value)

# for k,v in depth_class.items():
#     (k for k,v in class_id)
    
    
#     print(k)
# #     print('v:', v)
# # #     if class_id = v
# # #     dclass = k
# # #     print(dclass)
# # print(class_id)

In [ ]:
# def parse_veg_class(site_id):
#     # Classifying by vegetation coverage 
#     veg_class = {'treeless':[1, 3], 'sparse':[4, 6], 'dense':[7, 9]}
   
#     vclass = None 
    
#     class_id = site_id[0]
    
#     if class_id.isnumeric():
#         class_id = int(class_id)

#         for k,v in veg_class.items():

#             if class_id >= v[0] and class_id <= v[1]:
#                 vclass = k 
                
#     return vclass

# def parse_depth_class(site_id):
#     # Classifying by expected depth 
#     depth_class = {'shallow':[1, 4, 7], 'medium':[2, 5, 8], 'deep':[3, 6, 9]} #[1:3:9]
   
#     dclass = None 
    
#     class_id = site_id[0]
    
#     if class_id.isnumeric(): #for the outlier TS site
#         class_id = int(class_id) #cast as integer

#         for k,v in depth_class.items(): #for the key, value pairs in the dict listed above:

#             if class_id = v[0] or v[1] or v[2]:
#                 dclass = k 
#                 break

#     return dclass 

# site_id = '5N34'
# dclass = None 
# class_id = site_id[0]

# # (k for k,v in dict.iteritems() if v == value)

# for k,v in depth_class.items():
#     (k for k,v in class_id)
    
    
#     print(k)
# #     print('v:', v)
# # #     if class_id = v
# # #     dclass = k
# # #     print(dclass)
# # print(class_id)

# # load the database
# db_name = 'snow:hackweek@52.32.183.144/snowex'
# engine, session = get_db(db_name)

# result = session.query(LayerData.type).distinct().all()
# print(result)

# qry = session.query(LayerData).filter(LayerData.type=='density')

# # Form our dataframe from the query 
# df = query_to_geopandas(qry, engine)

# # Form a handy lambda for performant ops in df's
# veg_class_lambda = lambda row: parse_veg_class(row['site_id'])
# df['veg_class'] = df.apply(veg_class_lambda, axis=1)

# # Show off our df 
# df.plot()